Purpose of this notebook is to run through the logic of one step through the battery environment

Tests to implement
-  battery empty, charge 
-  battery full, charge 

-  battery just about empty, charge
-  battery just about full, charge

In [1]:
import numpy as np

from energy_py.main.scripts.spaces import Continuous_Space, Discrete_Space

In [2]:
#  create a function that mimics the step
def step(old_charge, action_space, eletricity_price, action):
    
    round_trip_eff = 0.8
    #  check the action is of the correct shape
    assert action.shape == (1, 2)
    
    #  our action is sent to the environment as (1, num_actions)
    #  we pull out the action here to make the code below cleaner
    action = action[0]
    
    #  checking the actions are valid
    
    for i, act in enumerate(action):
        assert act >= action_space[i].low
        assert act <= action_space[i].high
        
    #  calculate the net effect of the two actions
    #  also convert from MW to MWh/5 min by /12
    net_charge = (action[0] - action[1]) / 12
    net_charge = net_charge
    
    #  we first check to make sure this charge is within our capacity limits
    unbounded_new_charge = old_charge + net_charge
    bounded_new_charge = max(min(unbounded_new_charge, capacity), 0)

    #  we can now calculate the gross rate of charge or discharge
    gross_rate = (bounded_new_charge - old_charge) * 12
    
    #  now we account for losses
    if gross_rate > 0:
        #  we lose electricity when we charge
        losses = gross_rate * (1 - round_trip_eff) / 12
    else:
        #  we don't lose anything when we discharge
        losses = 0

    #  we can now calculate the new charge of the battery after losses
    new_charge = old_charge + gross_rate / 12 - losses
    #  this allows us to calculate how much electricity we actually store
    net_stored = new_charge - old_charge
    #  and to calculate our actual rate of charge or discharge
    net_rate = net_stored * 12    
    
    #  set a tolerance for the energy balances
    tolerance = 1e-10

    #  energy balance
    assert (new_charge) - (old_charge + net_stored) < tolerance
    #  check that our net_rate and net_stored are consistent
    assert (net_rate) - (12 * net_stored) < tolerance
    
    #  now we can calculate the reward
    #  the reward is simply the cost to charge
    #  or the benefit from discharging
    #  note that we use the gross rate - this is what the site incomer will see
    reward = -(gross_rate / 12) * electricity_price

    if verbose > 0:
        print('action was {}'.format(action))
        print('old charge was {} MWh'.format(old_charge))
        print('new charge is {} MWh'.format(new_charge))
        print('gross rate is {} MW'.format(gross_rate))
        print('losses were {} MWh'.format(losses))
        print('net rate is {} MW'.format(net_rate))        
        print('reward is {} $/5min'.format(reward))
    
    reward = 0
    return reward, new_charge

In [3]:
#  setup stuff that is an attribute of the Battery_Env class
electricity_price = float(30)

#  the battery rating & capacity
power_rating = 2.0
capacity = 3.0
old_charge = 0.1
verbose = 1

#  we also need an action space
#  this is set in the Battery_Env class
action_space = [Continuous_Space(low  = 0,
                                 high = power_rating),
                Continuous_Space(low  = 0,
                                 high = power_rating)]


In [4]:
#  we create an action - note that this is a single action of dimension 2
action = np.array([0,2]).reshape(1,-1)
action

array([[0, 2]])

In [5]:
#  simulate one step through the environment
rew, new_charge = step(old_charge, action_space, electricity_price, action)

action was [0 2]
old charge was 0.1 MWh
new charge is -1.3877787807814457e-17 MWh
gross rate is -1.2000000000000002 MW
losses were 0 MWh
net rate is -1.2000000000000002 MW
reward is 3.0000000000000004 $/5min
